In [1]:
%pip install sentencepiece transformers datasets torch scikit-learn
%pip install nepalitokenizers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
import os
import sentencepiece as spm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, DatasetDict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
# Step 1: Train SentencePiece Tokenizer
def train_sentencepiece_tokenizer(data_file, model_prefix, vocab_size=32000):
    """
    Train a SentencePiece tokenizer for the Nepali language.
    """
    spm.SentencePieceTrainer.train(
        input=data_file,
        model_prefix=model_prefix,
        vocab_size=vocab_size,
        model_type="bpe",  # Can be "unigram", "bpe", etc.
        character_coverage=0.9995,  # Adjust for Devanagari
        pad_id=0,
        unk_id=1,
        bos_id=2,
        eos_id=3
    )
    print(f"SentencePiece model trained and saved as {model_prefix}.model")


In [4]:
import csv
import random

input_file = 'chunkeddataset.txt'
train_file = 'nepali_train.csv'
test_file = 'nepali_test.csv'
test_size_ratio = 0.2  # 20% for testing

# Process the text file and split into train/test datasets
with open(input_file, 'r', encoding='utf-8') as infile, \
     open(train_file, 'w', encoding='utf-8', newline='') as train_out, \
     open(test_file, 'w', encoding='utf-8', newline='') as test_out:
    
    train_writer = csv.writer(train_out)
    test_writer = csv.writer(test_out)
    
    # Write headers
    train_writer.writerow(['text'])
    test_writer.writerow(['text'])
    
    # Read and process line by line
    for line in infile:
        line = line.strip()
        if line:  # Skip empty lines
            if random.random() < test_size_ratio:
                test_writer.writerow([line])
            else:
                train_writer.writerow([line])


                



In [5]:
file_path = 'nepali_train.csv'

with open(file_path, 'r', encoding='utf-8') as file:
    rows = list(csv.reader(file))

# Filter out rows containing '------'
rows = [row for row in rows if '------' not in row]

# Overwrite the original file with cleaned data
with open(file_path, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)

print(f"Cleaned and overwritten {file_path}")


Cleaned and overwritten nepali_train.csv


In [6]:
file_path = 'nepali_test.csv'

with open(file_path, 'r', encoding='utf-8') as file:
    rows = list(csv.reader(file))

# Filter out rows containing '------'
rows = [row for row in rows if '------' not in row]

# Overwrite the original file with cleaned data
with open(file_path, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)

print(f"Cleaned and overwritten {file_path}")


Cleaned and overwritten nepali_test.csv


In [7]:
# Step 2: Load Dataset
def load_nepali_dataset():
    """
    Load or create a Nepali dataset for the classification task.
    """
    # Replace this with actual data loading logic
    dataset = load_dataset("csv", data_files={"train": "nepali_train.csv", "test": "nepali_test.csv"})
    return DatasetDict({"train": dataset["train"], "test": dataset["test"]})

In [8]:
import pickle
from nepalitokenizers import WordPiece, SentencePiece
from datasets import Dataset


In [9]:

# Initialize tokenizers
tokenizer_wp = WordPiece()
tokenizer_sp = SentencePiece()
file_path='chunkeddataset.txt'

In [10]:

# Function to load Nepali text from a .txt file
def load_nepali_text_file(file_path):
    """
    Load Nepali text from a .txt file and return a list of sentences.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        # Read all lines from the file
        lines = f.readlines()
    
    # Clean up the lines: Remove any unnecessary newlines and empty lines
    sentences = [line.strip() for line in lines if line.strip()]
    return sentences


In [11]:

# Function to tokenize a dataset using a specified tokenizer
# Function to tokenize a dataset using a specified tokenizer
def tokenize_dataset(sentences, tokenizer):
    """
    Tokenizes each Nepali sentence using the specified tokenizer.
    """
    def tokenize_fn(text):
        # Tokenizing the text using the provided tokenizer
        tokens = tokenizer.encode(text)
        return {
            "ids": tokens.ids,
            "tokens": tokens.tokens
        }

    # Tokenize each sentence and collect input_ids and tokens
    tokenized_data = [tokenize_fn(sentence) for sentence in sentences]

    # Separate ids and tokens into their respective lists
    input_ids = [item['ids'] for item in tokenized_data]
    tokens = [item['tokens'] for item in tokenized_data]

    return input_ids, tokens


In [12]:

# Function to save tokenized data as .pkl
def save_as_pkl(data, file_name):
    with open(file_name, 'wb') as f:
        pickle.dump(data, f)

# Function to save vocabulary as .vocab
def save_vocab(vocab, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for token in vocab:
            f.write(f"{token}\n")


In [13]:


# Load Nepali text from a .txt file
sentences = load_nepali_text_file(file_path)

# Tokenize the dataset using SentencePiece
input_ids_sp, tokens_sp = tokenize_dataset(sentences, tokenizer_sp)

# Save tokenized data as .pkl
save_as_pkl({
    "input_ids": input_ids_sp,
    "tokens": tokens_sp
}, "sentencepiece_tokenized.pkl")

# Save SentencePiece vocabulary
sp_vocab = tokenizer_sp.get_vocab()  # Assuming get_vocab returns a list of vocabulary tokens
save_vocab(sp_vocab, "sentencepiece_vocab.vocab")

# Tokenize the dataset using WordPiece
input_ids_wp, tokens_wp = tokenize_dataset(sentences, tokenizer_wp)

# Save tokenized data as .pkl
save_as_pkl({
    "input_ids": input_ids_wp,
    "tokens": tokens_wp
}, "wordpiece_tokenized.pkl")

# Save WordPiece vocabulary
wp_vocab = tokenizer_wp.get_vocab()  # Assuming get_vocab returns a list of vocabulary tokens
save_vocab(wp_vocab, "wordpiece_vocab.vocab")

# Optionally, display a confirmation message
print("Tokenized data and vocab files saved successfully.") 


Tokenized data and vocab files saved successfully.


In [14]:
print("SentencePiece Tokenized IDs:", input_ids_sp[:5])
print("SentencePiece Tokens:", tokens_sp[:5])
print("WordPiece Tokenized IDs:", input_ids_wp[:5])
print("WordPiece Tokens:", tokens_wp[:5])


SentencePiece Tokenized IDs: [[35, 43, 1721, 14113, 9, 215, 2332, 7, 3591, 13277, 12, 3218, 25, 8962, 8652, 2894, 1028, 9169, 13, 7352, 1298, 1771, 111, 16441, 336, 135, 7, 24020, 8425, 2528, 687, 2189, 8425, 2014, 6789, 349, 9853, 768, 1771, 858, 1548, 2622, 26, 2216, 8425, 10, 1791, 43, 7690, 11002, 8992, 797, 10639, 8425, 2332, 36, 402, 203, 20938, 21659, 14713, 10, 44, 6870, 402, 1, 0], [7, 199, 2014, 6789, 349, 9853, 768, 1771, 858, 1548, 2622, 26, 2216, 8425, 10, 1791, 43, 7690, 11002, 8992, 797, 10639, 8425, 2332, 36, 402, 203, 20938, 21659, 14713, 10, 44, 6870, 402, 1, 0], [7, 199, 2014, 6789, 349, 9853, 768, 1771, 858, 1548, 2622, 26, 2216, 8425, 10, 1791, 43, 7690, 11002, 8992, 797, 10639, 8425, 2332, 36, 402, 203, 20938, 21659, 14713, 10, 44, 6870, 402, 119, 7690, 11002, 1881, 145, 349, 1869, 3758, 9, 2121, 22, 46, 31, 402, 203, 14810, 3758, 4799, 367, 130, 47, 7, 24020, 8425, 10, 13, 1, 0], [119, 7690, 11002, 1881, 145, 349, 1869, 3758, 9, 2121, 22, 46, 31, 402, 203, 14810,

In [15]:
# Step 4: Define Metrics for Evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [17]:
def train_transformer_model(dataset, tokenizer, model_name="xlm-roberta-base", num_labels=2):
    """
    Fine-tune a transformer-based model for Nepali text classification.
    """
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        save_strategy="epoch",
        logging_dir="./logs",
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    trainer.train()
    print("Training complete!")
    return model

In [18]:
# Run the pipeline
from nepali_tokenizer import NepaliTokenizer
if __name__ == "__main__":
    # File paths
    nepali_text_file = "C:/Users/L E G I O N/Desktop/Nepali-LLM/data/merged_nepali_corpus.txt"  # Path to a file with Nepali sentences
    spm_model_prefix = "nepali_spm"

    # Train SentencePiece Tokenizer
    train_sentencepiece_tokenizer(nepali_text_file, spm_model_prefix)

    # Load Dataset
    dataset = load_nepali_dataset()

    # Tokenize Dataset
  

from datasets import Dataset

# Initialize the NepaliTokenizer
tokenizer = NepaliTokenizer()

# Function to load Nepali sentences from a .txt file
def load_nepali_text_file(nepali_text_file):
    """
    Load Nepali text from a .txt file and return a list of sentences.
    """
    with open(nepali_text_file, 'r', encoding='utf-8') as f:
        # Read all lines from the file
        lines = f.readlines()
    
    # Clean up the lines: Remove any unnecessary newlines and empty lines
    sentences = [line.strip() for line in lines if line.strip()]
    return sentences

# Function to tokenize the dataset using NepaliTokenizer
def tokenize_dataset(sentences):
    """
    Tokenizes each Nepali sentence using NepaliTokenizer.
    """
    def tokenize_fn(text):
        # Tokenizing the text using NepaliTokenizer
        return tokenizer.tokenizer(text)
    
    # Tokenizing each sentence
    tokenized_sentences = [tokenize_fn(sentence) for sentence in sentences]
    
    return tokenized_sentences

# Load Nepali text from your local .txt file

sentences = load_nepali_text_file(nepali_text_file)

# Tokenize the sentences using NepaliTokenizer
tokenized_sentences = tokenize_dataset(sentences)

# Convert the tokenized data into a Hugging Face Dataset format
dataset = Dataset.from_dict({"text": sentences, "input_ids": tokenized_sentences})

# Optionally, display the first few entries of the dataset
print(dataset[:5])  # Print first 5 tokenized entries

#     # # Train Transformer Model
# trained_model = train_transformer_model(dataset, tokenizer)

ModuleNotFoundError: No module named 'nepali_tokenizer'

In [32]:
import pickle

# Path to your .pkl file
file_path = 'C:/Users/L E G I O N/Desktop/Nepali-LLM/Preprocessing/wordpiece_tokenized.pkl'

# Open and load the file
with open(file_path, 'rb') as f:
    data = pickle.load(f)

# Print the type of the loaded object
print(f"Loaded data type: {type(data)}")

# Inspect the data
if isinstance(data, list):
    print(f"List with {len(data)} elements. First element:")
    print(data[0])
elif isinstance(data, dict):
    print(f"Dictionary with keys: {list(data.keys())}")
    # Example: Inspect the 'train' key if it exists
    if 'input_ids' in data:
        print(f"'train' key contains {len(data['input_ids'])} sequences. First sequence:")
        print(data['input_ids'][0])
else:
    print("Loaded data:")
    print(data)


Loaded data type: <class 'dict'>
Dictionary with keys: ['input_ids', 'tokens']
'train' key contains 77044 sequences. First sequence:
[1, 8452, 8472, 10744, 8445, 10488, 8334, 5748, 9313, 13033, 5713, 15959, 8340, 14427, 8444, 11867, 1496, 8691, 13459, 10683, 5792, 5796, 16, 11341, 5723, 8892, 8429, 13816, 9802, 8371, 8425, 1496, 10072, 25694, 5712, 20967, 5757, 5763, 10478, 15925, 5712, 20967, 8392, 8585, 9267, 17488, 9279, 8892, 8492, 5757, 11339, 19392, 20967, 8337, 12152, 8472, 13896, 15893, 21336, 9148, 20967, 9313, 1496, 9640, 9074, 12487, 11450, 28511, 28072, 11098, 8511, 11322, 1496, 9640, 2]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import numpy as np
from torch.nn.utils.rnn import pad_sequence

# Enable anomaly detection for debugging in-place errors
torch.autograd.set_detect_anomaly(True)

# Load tokenized data and Word2Vec embeddings
with open('wordpiece_tokenized.pkl', 'rb') as f:
    tokenized_data = pickle.load(f)

# Ensure that we have 'input_ids' and 'tokens' in the data
input_ids = tokenized_data.get('input_ids', [])
tokens = tokenized_data.get('tokens', [])

if not input_ids or not tokens:
    raise ValueError("The tokenized data does not contain 'input_ids' or 'tokens'.")

# Load Word2Vec embeddings
word2vec_path = "processed_normalized.word2vec.wv.vectors.npy"
word2vec = np.load(word2vec_path)

# Create embedding matrix
vocab_file = 'wordpiece_vocab.vocab'

# Read the vocabulary file and create a mapping
vocab = {}
idx_to_word = {}
with open(vocab_file, 'r', encoding='utf-8') as f:
    for idx, line in enumerate(f):
        token = line.strip()  # Remove any extra whitespace
        vocab[token] = idx
        idx_to_word[idx] = token

# Get the vocabulary size
vocab_size = len(vocab)
print(f"Vocabulary Size: {vocab_size}")

embedding_dim = word2vec.shape[1]
embedding_matrix = torch.tensor(word2vec, dtype=torch.float32)

# Model: Simple RNN-based Language Model
class SmallTextGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(SmallTextGenerator, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        embeds = self.embedding(x)
        output, hidden = self.rnn(embeds, hidden)
        logits = self.fc(output)
        return logits, hidden

# Hyperparameters
hidden_dim = 128
num_layers = 2
learning_rate = 0.001
batch_size = 32
num_epochs = 10

# Initialize model, loss, and optimizer
model = SmallTextGenerator(vocab_size, embedding_dim, hidden_dim, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Prepare data for training
def get_batches(data, batch_size):
    for i in range(0, len(data) - batch_size + 1, batch_size):
        batch = data[i:i+batch_size]
        # Pad sequences to the same length within a batch
        padded_batch = pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in batch], batch_first=True)
        yield padded_batch

# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    hidden = None
    for batch in get_batches(input_ids, batch_size):
        inputs = batch[:, :-1]  # All but the last token
        targets = batch[:, 1:]  # All but the first token

        optimizer.zero_grad()
        
        # Forward pass
        outputs, hidden = model(inputs, hidden)

        # Print the shape of outputs and targets for debugging
        print(f"Epoch {epoch+1}, Batch shape debug:")
        print(f"Outputs shape: {outputs.shape}")
        print(f"Targets shape: {targets.shape}")

        # Reshape the outputs and targets to 1D for CrossEntropyLoss
        outputs_reshaped = outputs.contiguous().view(-1, vocab_size)
        targets_reshaped = targets.contiguous().view(-1)

        # Check the reshaped shapes
        print(f"Reshaped Outputs shape: {outputs_reshaped.shape}")
        print(f"Reshaped Targets shape: {targets_reshaped.shape}")

        # Ensure the reshaped tensors match
        assert outputs_reshaped.size(0) == targets_reshaped.size(0), "Shape mismatch between outputs and targets."

        # Calculate loss
        loss = criterion(outputs_reshaped, targets_reshaped)
        
        # Backward pass
        loss.backward()  # Removed retain_graph=True
        optimizer.step()

        # Clear hidden states to avoid holding the computational graph
        hidden = None

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")

# Text generation function
def generate_text(model, start_token, max_length=50, temperature=1.0):
    model.eval()
    generated = [start_token]
    input_token = torch.tensor([start_token], dtype=torch.long).unsqueeze(0)
    hidden = None

    for _ in range(max_length):
        output, hidden = model(input_token, hidden)
        output = output.squeeze(0).div(temperature).exp()
        next_token = torch.multinomial(output, num_samples=1).item()
        generated.append(next_token)
        input_token = torch.tensor([[next_token]], dtype=torch.long)

        if next_token == vocab.get('<EOS>', -1):  # Check for EOS token
            break

    return ' '.join([idx_to_word.get(idx, "<UNK>") for idx in generated])

# Save the trained model
torch.save(model.state_dict(), "small_nepali_text_generator.pt")


Vocabulary Size: 30522
Epoch 1, Batch shape debug:
Outputs shape: torch.Size([32, 74, 30522])
Targets shape: torch.Size([32, 74])
Reshaped Outputs shape: torch.Size([2368, 30522])
Reshaped Targets shape: torch.Size([2368])
Epoch 1, Batch shape debug:
Outputs shape: torch.Size([32, 93, 30522])
Targets shape: torch.Size([32, 93])
Reshaped Outputs shape: torch.Size([2976, 30522])
Reshaped Targets shape: torch.Size([2976])
Epoch 1, Batch shape debug:
Outputs shape: torch.Size([32, 71, 30522])
Targets shape: torch.Size([32, 71])
Reshaped Outputs shape: torch.Size([2272, 30522])
Reshaped Targets shape: torch.Size([2272])
Epoch 1, Batch shape debug:
Outputs shape: torch.Size([32, 79, 30522])
Targets shape: torch.Size([32, 79])
Reshaped Outputs shape: torch.Size([2528, 30522])
Reshaped Targets shape: torch.Size([2528])
Epoch 1, Batch shape debug:
Outputs shape: torch.Size([32, 78, 30522])
Targets shape: torch.Size([32, 78])
Reshaped Outputs shape: torch.Size([2496, 30522])
Reshaped Targets sha